# EDA

In [2]:
import os
import sys
import polars as pl
import numpy as np
#polarsの表示limitをなくす
pl.Config.set_tbl_cols(-1)

sys.path.append(os.path.join('..'))
from exp.features import Features, FeatureSelecter
from exp.models.cat import CatBoostModel
from exp.models.lgbm import LGBMModel
from exp.models.rf import RandomForestModel
from exp.models.rgf import RGFModel
from exp.models.xgb import XGBModel
from exp.models.lr import LinerRegressionModel

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import gc
gc.enable()

/home/education/signate_scup2023/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
train = pl.read_csv('../input/train.csv')
test = pl.read_csv('../input/test.csv')

In [4]:
features = Features(train, test)
train, test = features.create_features()

train.head(2)

shape: (2, 491)
┌─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┐
│ id  ┆ reg ┆ yea ┆ man ┆ con ┆ cyl ┆ fue ┆ odo ┆ tit ┆ tra ┆ dri ┆ siz ┆ typ ┆ pai ┆ sta ┆ pri ┆ odo ┆ age ┆ odo ┆ odo ┆ lat ┆ lng ┆ lat ┆ lng ┆ lat ┆ man ┆ con ┆ fue ┆ tit ┆ tra ┆ dri ┆ siz ┆ typ ┆ pai ┆ sta ┆ man ┆ con ┆ fue ┆ tit ┆ tra ┆ dri ┆ siz ┆ typ ┆ pai ┆ sta ┆ typ ┆ typ ┆ typ ┆ man ┆ man ┆ man ┆ typ ┆ typ ┆ typ ┆ typ ┆ typ ┆ typ ┆ typ ┆ man ┆ fue ┆ tit ┆ tra ┆ dri ┆ typ ┆ pai ┆ sta ┆ lat ┆ tit ┆ sta ┆ reg ┆ pai ┆ lat ┆ con ┆ siz ┆ man ┆ man ┆ man ┆ man ┆ man ┆ man ┆ man ┆ man ┆ man ┆ man ┆ con ┆ con ┆ con ┆ con ┆ con ┆ con ┆ con ┆ con ┆ con ┆ con ┆ cyl ┆ cyl ┆ cyl ┆ cyl ┆ cyl ┆ cyl ┆ cyl ┆ cyl ┆ cyl ┆ cyl ┆ fue ┆ fue ┆ fue ┆ fue ┆ fue ┆ fue ┆ fue ┆ fue ┆ fue ┆ fue ┆ dri ┆ dri ┆ dri ┆ dri ┆ dri ┆ dri ┆ dri ┆ dri ┆ dri ┆ dri ┆ typ ┆ typ ┆ typ ┆ typ ┆ typ ┆ typ ┆ typ ┆ typ ┆ typ ┆ typ ┆ car ┆ car ┆ car ┆ car ┆ car ┆ car ┆ car ┆ car ┆ car ┆ car ┆ car ┆ car ┆ car ┆ car ┆ car ┆ car ┆ car ┆ car ┆ car ┆ car ┆ car ┆ car ┆ car ┆ car ┆ car ┆ car ┆ car ┆ car ┆ car ┆ car ┆ car ┆ car ┆ car ┆ car ┆ car ┆ car ┆ car ┆ car ┆ ca

In [5]:
list(set(train["manufacturer"].unique().to_list() + test["manufacturer"].unique().to_list()))

['jaguar',
 'aston_martin',
 'audi',
 'land_rover',
 'infiniti',
 'acura',
 'buick',
 'hyundai',
 'chevrolet',
 'toyota',
 'volkswagen',
 'mitsubishi',
 'honda',
 'alfa_romeo',
 'ford',
 'ram',
 'subaru',
 'mercedes_benz',
 'mazda',
 'porsche',
 'fiat',
 'jeep',
 'bmw',
 'chrysler',
 'volvo',
 'gmc',
 'dodge',
 'kia',
 'mercury',
 'saturn',
 'rover',
 'nissan',
 'pontiac',
 'mini',
 'lexus',
 'lincoln',
 'cadillac']

In [7]:
print(set(test["manufacturer"].unique().to_list()) - set(train["manufacturer"].unique().to_list()))
print(set(train["manufacturer"].unique().to_list()) - set(test["manufacturer"].unique().to_list()))

set()
set()


In [ ]:
df = pl.concat([train, test], how="diagonal")

df.head()

## region

ユニーク数が300近くあるので、onehotは適さないかも

自然言語を用いたアプローチを考えるべき？

In [ ]:
uq_region_train = set(train["region"]) - set(test["region"])
uq_region_test = set(test["region"]) - set(train["region"])
print("Unique regions in train: ", uq_region_train)
print("Unique regions in test: ", uq_region_test)

In [ ]:
df["region"].value_counts(sort=True)

## year

minは1918
maxは2022
よって基準年は2023年と仮定できる

yearの異常値は四桁目を間違えたtypoの可能性が高い
->2030以上は1000引く

ageは古くなると逆に価値が上がる。conditionと踏まえた特徴量を作るといいかも

In [ ]:
sns.histplot(df["year"])

In [ ]:
sns.histplot(test["year"])

In [ ]:
sns.barplot(x="age", y="price", data=df.to_pandas())

## manufacturer

サンプル数が少ないものもあるので、otherとかにしたほうがいいかも

In [ ]:
df["manufacturer"].value_counts(sort=True)
#リストをprint
print(df["manufacturer"].value_counts(sort=True).index.tolist())

In [ ]:
ax = sns.barplot(x="manufacturer", y="price", data=df.to_pandas())
plt.xticks(rotation=90)
plt.show()

## condition
これはlabelencodingでいけそう

New
Like new
Excellent
Good
Salvage
Fair

In [ ]:
df["condition"].value_counts(sort=True)

In [ ]:
sns.barplot(x="condition", y="price", data=df.to_pandas())

## cyclinders

現在の軽自動車は3気筒、乗用車では4〜8気筒が主流となっていますが、大型車やスポーツカーでは気筒数が多く、排気量も大きくなります。

In [ ]:
df["cylinders"].value_counts()

In [ ]:
sns.barplot(x="cylinders", y="price", data=df.to_pandas())

## fuel

In [ ]:
df["fuel"].value_counts(sort=True)

In [ ]:
sns.barplot(x="fuel", y="price", data=df.to_pandas())

# odometer

In [ ]:
df["odometer"].describe()

In [ ]:
sns.histplot(df["odometer"], kde=True)

## title_status

In [ ]:
df["title_status"].value_counts(sort=True)

In [ ]:
sns.barplot(x="title_status", y="price", data=df.to_pandas())

In [ ]:
df.filter(pl.col("title_status") == "nan")

## transmission

In [ ]:
df["transmission"].value_counts()

In [ ]:
sns.barplot(x="transmission", y="price", data=df.to_pandas())

## drive

In [ ]:
df["drive"].value_counts()

In [ ]:
sns.barplot(x="drive", y="price", data=df.to_pandas())

## size

In [ ]:
df["size"].value_counts()

In [ ]:
sns.barplot(x="size", y="price", data=df.to_pandas())

## type

In [ ]:
df["type"].value_counts(sort=True)

In [ ]:
sns.barplot(x="type", y="price", data=df.to_pandas())
plt.xticks(rotation=90)
plt.show()

In [ ]:
df.filter(pl.col("type") == "nan")

## paint_color

In [ ]:
df["paint_color"].value_counts()

In [ ]:
sns.barplot(x="paint_color", y="price", data=df.to_pandas())
plt.xticks(rotation=90)
plt.show()

## state

In [ ]:
df["state"].value_counts(sort=True)

In [ ]:
sns.barplot(x="state", y="price", data=df.to_pandas())
plt.xticks(rotation=90)
plt.show()

## price

In [ ]:
sns.histplot(df["price"], kde=True)

manufacturerごとのtypeの数一覧

In [ ]:
pl.Config.set_tbl_rows(-1)
df_type = df.to_pandas()
df_type.pivot_table(index='manufacturer', columns='type', values='id', aggfunc='count', fill_value=0)